In [2]:
import numpy as np
import scipy.linalg as la
import math
import dimod
from dimod import SimulatedAnnealingSampler

In [3]:
def generate_quip(n=5, kappa=50, sparsity=0.5, seed=42):
    """Generate a convex QUIP instance with known optimal solution"""
    np.random.seed(seed)
    U2 = np.array([-2, -1, 0, 1, 2])
    Q_initial = np.random.choice(U2, size=(n, n))
    Q_initial = (Q_initial + Q_initial.T) / 2
    sparsity_mask = np.random.random((n, n)) < sparsity
    Q_initial = Q_initial * sparsity_mask
    
    for i in range(n):
        if Q_initial[i, i] == 0:
            Q_initial[i, i] = np.random.choice([1, 2])
    
    eigenvalues = la.eigvals(Q_initial)
    lambda_min = np.min(eigenvalues)
    lambda_add = np.ceil((abs(min(lambda_min, 0))) + np.random.random())
    Q = Q_initial + lambda_add * np.eye(n)
    
    eigenvalues_final = la.eigvals(Q)
    x_star = np.zeros(n, dtype=int)
    num_nonzero = np.random.randint(1, n)
    nonzero_indices = np.random.choice(n, size=num_nonzero, replace=False)
    for i in nonzero_indices:
        x_star[i] = np.random.randint(1, kappa + 1)
    
    q = -2 * Q @ x_star
    kappa_vec = np.full(n, kappa, dtype=int)
    
    objective_at_x_star = x_star.T @ Q @ x_star + q.T @ x_star
    gradient_at_x_star = 2 * Q @ x_star + q
    
    print(f"Gradient at x* (should be ~0): {gradient_at_x_star}")
    return Q, q, kappa_vec, x_star

In [4]:
def create_binary_encoder(kappa):
    """Create binary encoding parameters for given upper bound"""
    width = math.floor(math.log2(kappa)) + 1
    coefficients = [2**i for i in range(width)]
    return width, coefficients

In [5]:
def encode_integer(value, width, coefficients):
    """Encode single integer to binary using coefficients"""
    binary = []
    remaining = value
    for coeff in reversed(coefficients):
        if remaining >= coeff:
            binary.append(1)
            remaining -= coeff
        else:
            binary.append(0)
    binary.reverse()
    return binary

In [6]:
def decode_binary(binary, coefficients):
    """Decode binary representation back to integer"""
    return sum(bit * coeff for bit, coeff in zip(binary, coefficients))

In [7]:
def encode_vector(integer_vector, kappa):
    """Encode full integer vector to binary"""
    width, coefficients = create_binary_encoder(kappa)
    
    binary_vector = []
    for val in integer_vector:
        if val < 0 or val > kappa:
            raise ValueError(f"Value {val} outside bounds [0, {kappa}]")
        binary_repr = encode_integer(val, width, coefficients)
        binary_vector.extend(binary_repr)
    
    return binary_vector, width, coefficients

In [8]:
def decode_vector(binary_vector, n_variables, width, coefficients):
    """Decode binary vector back to integers"""
    integer_vector = []
    
    for i in range(n_variables):
        start = i * width
        end = start + width
        var_binary = binary_vector[start:end]
        integer_val = decode_binary(var_binary, coefficients)
        integer_vector.append(integer_val)
    
    return integer_vector

In [9]:
def create_encoding_matrix(n_variables, width, coefficients):
    """Create encoding matrix C where x = C @ y"""
    total_binary_vars = width * n_variables
    C = np.zeros((n_variables, total_binary_vars))
    
    for i in range(n_variables):
        start = i * width
        end = start + width
        C[i, start:end] = coefficients
    
    return C

In [10]:
def convert_to_qubo(Q_binary, q_binary):
    """
    Convert quadratic problem min y^T Q_binary y + q_binary^T y to QUBO format
    QUBO format: min y^T Q_qubo y where linear terms are on the diagonal
    """
    n = Q_binary.shape[0]
    Q_qubo = Q_binary.copy()
    
    # Add linear terms to diagonal
    for i in range(n):
        Q_qubo[i, i] += q_binary[i]
    
    return Q_qubo

In [11]:
Q, q, kappa_vec, x_star = generate_quip(n = 3, kappa = 25)
print(f"Known optimal solution: {x_star}")
print(f"Optimal objective value: {(x_star.T @ Q @ x_star + q.T @ x_star):.4f}")

Gradient at x* (should be ~0): [0. 0. 0.]
Known optimal solution: [ 0 10  0]
Optimal objective value: -300.0000


In [12]:
width, coefficients = create_binary_encoder(kappa = 25)
coefficients

[1, 2, 4, 8, 16]

In [13]:
C = create_encoding_matrix(3, width, coefficients)
C.shape

(3, 15)

In [14]:
Q_binary = C.T @ Q @ C
q_binary = C.T @ q

Q_binary.shape, q_binary.shape

((15, 15), (15,))

In [15]:
Q_qubo = convert_to_qubo(Q_binary, q_binary)
Q_qubo.shape

(15, 15)

In [16]:
bqm = dimod.BQM.from_qubo(Q_qubo)

In [17]:
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm, num_reads = 100)

In [18]:
best_sample = sampleset.first
best_energy = best_sample.energy

In [19]:
n_binary_vars = width * 3
binary_solution = [0] * n_binary_vars
for var_idx, value in best_sample.sample.items():
    binary_solution[var_idx] = value

binary_solution

[np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(1),
 np.int8(0),
 np.int8(1),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0),
 np.int8(0)]

In [20]:
integer_solution = np.array(decode_vector(binary_solution, 3, width, coefficients))
original_objective = integer_solution.T @ Q @ integer_solution + q.T @ integer_solution

integer_solution

array([ 0, 10,  0], dtype=int8)

In [21]:
C

array([[ 1.,  2.,  4.,  8., 16.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  2.,  4.,  8., 16.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  2.,  4.,
         8., 16.]])